<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=200077460" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2023-08-08 00:00:00,BASE27789-USD,0.000018,0.000016,0.000018,0.000015,61413.0,0.000016
1,2023-08-09 00:00:00,BASE27789-USD,0.000016,0.000019,0.000019,0.000013,142844.0,0.000019
2,2023-08-10 00:00:00,BASE27789-USD,0.000019,0.000027,0.000029,0.000016,219320.0,0.000027
3,2023-08-11 00:00:00,BASE27789-USD,0.000027,0.000058,0.000065,0.000025,1377180.0,0.000058
4,2023-08-12 00:00:00,BASE27789-USD,0.000058,0.000068,0.000088,0.000052,1470142.0,0.000068
...,...,...,...,...,...,...,...,...
703175,2024-10-04 00:00:00,RIDE-USD,0.003157,0.003307,0.003312,0.003148,13514.0,0.003307
703176,2024-10-05 00:00:00,RIDE-USD,0.003307,0.003219,0.003318,0.003213,6269.0,0.003219
703177,2024-10-06 00:00:00,RIDE-USD,0.003219,0.003634,0.003685,0.003218,29188.0,0.003634
703178,2024-10-07 00:00:00,RIDE-USD,0.003634,0.003457,0.003662,0.003395,15766.0,0.003457


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'
link

'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:09<00:00,  2.03it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:12<00:00,  1.57it/s]

2520


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1728412269


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2520/2520 [03:30<00:00, 11.97it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

678

# Read a sample

In [12]:
# jsonf = pd.read_json(f'/kaggle/working/crypto_data/BTC-USD.json')
# jsondict = jsonf.to_dict()
# timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
# sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
# symbol = list(sym for s in range(len(timestamp)))
# low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
# high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
# volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
# open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
# close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
# adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
# dates = return_timestamp(timestamp)
# tempdf = pd.DataFrame(
#     {
#     'dates':dates,
#     'symbol':symbol,
#     'open':open_,
#     'close':close,
#     'high':high,
#     'low':low,
#     'volume':volume,
#     'adj_close':adj_close
#     }
# )
# tempdf

# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list_dir):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


  7%|▋         | 47/678 [00:00<00:09, 63.35it/s]

Missing timestamp for CBPAY-USD .json. Skipping this symbol.
Missing timestamp for SUNCAT-USD .json. Skipping this symbol.
Missing timestamp for ONI32538-USD .json. Skipping this symbol.


 12%|█▏        | 78/678 [00:01<00:08, 66.71it/s]

Missing timestamp for UNIO-USD .json. Skipping this symbol.
Missing timestamp for FOFAR32727-USD .json. Skipping this symbol.


 14%|█▎        | 92/678 [00:01<00:10, 58.38it/s]

...

Missing timestamp for XR-USD .json. Skipping this symbol.


 88%|████████▊ | 596/678 [00:16<00:03, 22.61it/s]

Missing timestamp for SOY32350-USD .json. Skipping this symbol.


100%|██████████| 678/678 [00:19<00:00, 34.48it/s]

Missing timestamp for DLCBTC-USD .json. Skipping this symbol.


In [15]:
megadf['symbol'].unique()

array(['SUB6845-USD', 'SAKAI-USD', 'GAINS-USD', 'ABEL25232-USD',
       'BTX-USD', 'UTYAB-USD', 'VAB-USD', 'WAIT-USD', 'ASTA-USD',
       'ESE-USD', 'WARPED-USD', 'CRWNY-USD', 'MOOV-USD', 'WSI20848-USD',
       'LYNX-USD', 'ITHEUM-USD', 'CPU8295-USD', 'SENSO-USD', 'AEG-USD',
       'BLOCK30433-USD', 'FTC-USD', 'SOLCEX-USD', 'HVH-USD', 'DHB-USD',
       'GG28867-USD', 'BF-USD', 'BROCK-USD', 'NABOX-USD', 'VLXPAD-USD',
       'VALOR-USD', 'XMON-USD', 'ABYSS-USD', 'SNC-USD', 'BTCK-USD',
       'MANC-USD', 'DOGA-USD', 'PANW', 'FLAME-USD', 'PLX30038-USD',
       'STND-USD', 'SSG-USD', 'HIPENGUINS-USD', 'ADM-USD', 'SOS16463-USD',
       'REV3L-USD', 'PNP-USD', 'BEFI-USD', 'DOCK-USD', 'JETTON-USD',
       'VC28580-USD', 'LYM-USD', 'FO-USD', 'IDLE-USD', 'ILC-USD',
       'WALV-USD', 'NEURA-USD', 'CHI-USD', 'BASE27789-USD', 'IMGNAI-USD',
       'OOKS-USD', 'EMC29281-USD', 'INFRA-USD', 'OXBT-USD', 'WNT-USD',
       'HOSKY-USD', 'SQR-USD', 'MBD-USD', 'CVP-USD', 'ELON30979-USD',
       'WOKB-USD', 

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2023-12-06 00:00:00', '2024-10-08 18:28:00')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume       float64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume              float64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2023-12-06 00:00:00,SUB6845-USD,0.007990,0.008600,0.009106,0.007585,17524.0,0.008600
1,2023-12-07 00:00:00,SUB6845-USD,0.008589,0.008864,0.008954,0.008254,13064.0,0.008864
2,2023-12-08 00:00:00,SUB6845-USD,0.008864,0.009600,0.009603,0.008556,10829.0,0.009600
3,2023-12-09 00:00:00,SUB6845-USD,0.009600,0.010487,0.010859,0.009589,22847.0,0.010487
4,2023-12-10 00:00:00,SUB6845-USD,0.010487,0.009296,0.010857,0.009157,13906.0,0.009296
...,...,...,...,...,...,...,...,...
698698,2024-10-04 00:00:00,XPM-USD,0.027345,0.027922,0.028099,0.027212,0.0,0.027922
698699,2024-10-05 00:00:00,XPM-USD,0.027922,0.027320,0.027955,0.027151,94.0,0.027320
698700,2024-10-06 00:00:00,XPM-USD,0.027320,0.027638,0.028583,0.027278,8.0,0.027638
698701,2024-10-07 00:00:00,XPM-USD,0.027638,0.027398,0.028348,0.027381,11.0,0.027398


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2023-12-06 00:00:00,SUB6845-USD,0.007990,0.008600,0.009106,0.007585,17524.0,0.008600
1,2023-12-07 00:00:00,SUB6845-USD,0.008589,0.008864,0.008954,0.008254,13064.0,0.008864
2,2023-12-08 00:00:00,SUB6845-USD,0.008864,0.009600,0.009603,0.008556,10829.0,0.009600
3,2023-12-09 00:00:00,SUB6845-USD,0.009600,0.010487,0.010859,0.009589,22847.0,0.010487
4,2023-12-10 00:00:00,SUB6845-USD,0.010487,0.009296,0.010857,0.009157,13906.0,0.009296
...,...,...,...,...,...,...,...,...
698698,2024-10-04 00:00:00,XPM-USD,0.027345,0.027922,0.028099,0.027212,0.0,0.027922
698699,2024-10-05 00:00:00,XPM-USD,0.027922,0.027320,0.027955,0.027151,94.0,0.027320
698700,2024-10-06 00:00:00,XPM-USD,0.027320,0.027638,0.028583,0.027278,8.0,0.027638
698701,2024-10-07 00:00:00,XPM-USD,0.027638,0.027398,0.028348,0.027381,11.0,0.027398


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 87.1M/87.1M [00:01<00:00, 52.6MB/s]


Upload successful: 1000_cryptos.csv (87MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)